<a href="https://colab.research.google.com/github/dlwlgus53/KoreanEmbedding/blob/master/2_1%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 파일 읽기 

- wiki 백과
- koquard
- 네이버 영화 리뷰

파일을 읽고, 필요한 text를 추출하는 방법 코드입니다


#### 필요 모듈 load

In [ ]:
root_path = '/content/drive/MyDrive/2021-1/AI데이터활용교재개발/code'
import re, json, glob
from gensim.corpora import WikiCorpus, Dictionary
from gensim.utils import to_unicode


### Wiki 백과 파일에서 text 추출하기
- 데이터 : https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2

#### 정규표현식 지정

In [ ]:
WIKI_REMOVE_CHARS = re.compile("'+|(=+.{2,30}=+)|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):|\n", re.UNICODE)
WIKI_SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
WIKI_REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)


In [ ]:
def tokenize(content, token_min_len=2, token_max_len=100, lower=True):
    content = re.sub(EMAIL_PATTERN, ' ', content)  # remove email pattern
    content = re.sub(URL_PATTERN, ' ', content) # remove url pattern
    content = re.sub(WIKI_REMOVE_CHARS, ' ', content)  # remove unnecessary chars
    content = re.sub(WIKI_SPACE_CHARS, ' ', content)
    content = re.sub(MULTIPLE_SPACES, ' ', content)
    tokens = content.replace(", )", "").split(" ")
    result = []
    for token in tokens:
        if not token.startswith('_'):
            token_candidate = to_unicode(re.sub(WIKI_REMOVE_TOKEN_CHARS, '', token))
        else:
            token_candidate = ""
        if len(token_candidate) > 0:
            result.append(token_candidate)
    return result


In [ ]:
"""Convert Wikipedia xml dump file to text corpus"""
in_f = root_path + '/data/raw/kowiki-latest-pages-articles.xml.bz2' #
out_f = root_path + '/data/processed/processed_wiki_ko.txt' #

output = open(out_f, 'w', encoding = "utf-8")
wiki = WikiCorpus(in_f, tokenizer_func=tokenize, dictionary=Dictionary(), processes = 1)
i = 0
for text in wiki.get_texts():
    output.write(bytes(' '.join(text), 'utf-8').decode('utf-8') + '\n')
    i = i + 1
    if (i % 10000 == 0):
      print('Processed ' + str(i) + ' articles')
output.close()
print('Processing complete!')


Processed 10000 articles
Processed 20000 articles
Processed 30000 articles
Processed 40000 articles
Processed 50000 articles
Processed 60000 articles
Processed 70000 articles
Processed 80000 articles
Processed 90000 articles
Processed 100000 articles
Processed 110000 articles
Processed 120000 articles
Processed 130000 articles
Processed 140000 articles
Processed 150000 articles
Processed 160000 articles
Processed 170000 articles
Processed 180000 articles
Processed 190000 articles
Processed 200000 articles
Processed 210000 articles
Processed 220000 articles
Processed 230000 articles
Processed 240000 articles
Processed 250000 articles
Processed 260000 articles
Processed 270000 articles
Processed 280000 articles
Processed 290000 articles
Processed 300000 articles
Processed 310000 articles
Processed 320000 articles
Processed 330000 articles
Processed 340000 articles
Processed 350000 articles
Processed 360000 articles
Processed 370000 articles
Processing complete!


###  KorQuAD 
- 데이터 : https://github.com/korquad/korquad.github.io/tree/master/dataset 에서KorQuAD_v1.0_dev.json, KorQuAD_v1.0_train.json 다운로드


In [ ]:
def process_korQuAD(corpus_fname, output_fname):
    with open(corpus_fname) as f1, open(output_fname, 'w', encoding='utf-8') as f2:
        dataset_json = json.load(f1)
        dataset = dataset_json['data']
        for article in dataset:
            w_lines = []
            for paragraph in article['paragraphs']:
                w_lines.append(paragraph['context'])
                for qa in paragraph['qas']:
                    q_text = qa['question']
                    for a in qa['answers']:
                        a_text = a['text']
                        w_lines.append(q_text + " " + a_text)
            for line in w_lines:
                f2.writelines(line + "\n")


In [ ]:
# train파일 process
in_f_train = root_path + '/data/raw/KorQuAD_v1.0_train.json'
out_f_train = root_path + '/data/processed/processed_korquad_train.txt'
process_korQuAD(in_f_train, out_f_train)

# dev 파일 process
in_f_test = root_path + '/data/raw/KorQuAD_v1.0_dev.json'
out_f_test = root_path + '/data/processed/processed_korquad_dev.txt'
process_korQuAD(in_f_test, out_f_test)
# 두 파일을 합친 파일 생성
out_f_combined = root_path + '/data/processed/processed_korquad.txt'
with open(out_f_train, encoding='utf-8') as f1, open(out_f_test, encoding='utf-8') as f2, open(out_f_combined, 'w', encoding='utf-8') as f3:
    f3.write(f1.read() + '\n' + f2.read())


### 네이버 영화 리뷰 
데이터 : 
- https://github.com/e9t/nsmc/raw/master/ratings.txt
- https://github.com/e9t/nsmc/raw/master/ratings_train.txt
- https://github.com/e9t/nsmc/raw/master/ratings_test.txt


In [ ]:
def process_nsmc(corpus_path, output_fname, with_label=True, with_id = False):
  # file open with encoding
  with open(corpus_path, 'r', encoding='utf-8') as f1, \
                open(output_fname, 'w', encoding='utf-8') as f2:
            next(f1)  # skip head line
            for line in f1:
              # strip -> split
                id, sentence, label = line.strip().split('\t')
                if not sentence: continue
                if with_label:
                    f2.writelines(sentence + "\u241E" + label + "\n")
                elif with_id:
                    f2.writelines(sentence + "\u241E" + id + "\n")
                else:
                    f2.writelines(sentence + "\n")

In [ ]:
input_path = root_path + '/data/raw/ratings.txt'
output_path = root_path + '/data/processed/processed_ratings.txt'
process_nsmc(input_path, output_path, with_label=False, with_id = False)

input_path = root_path + '/data/raw/ratings.txt'
output_path = root_path + '/data/processed/processed_ratings_movieid.txt'
process_nsmc(input_path, output_path, with_label=False, with_id = True)

input_path = root_path + '/data/raw/ratings_train.txt'
output_path = root_path + '/data/processed/processed_ratings_train.txt'

process_nsmc(input_path, output_path, with_label=True, with_id = False)

input_path  =root_path + '/data/raw/ratings_test.txt'
output_path=  root_path + '/data/processed/processed_ratings_test.txt'
process_nsmc(input_path, output_path, with_label=True, with_id = False)



